In [149]:
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
import numpy as np
import os
import random
import nltk
import tensorflow as tf
import keras as krs
import json
import pickle
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt


Lmzr = WordNetLemmatizer()
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/Data Files/intents.json').read())

words = []
tags = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [152]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Data Files/intents.json'

with open(file_path, 'r', encoding='utf-8') as file:
    intents = json.load(file)

In [153]:
for tag in intents['intents']:
    for pattern in tag['patterns']:
        pattern_list = nltk.word_tokenize(pattern)
        words.extend(pattern_list)
        documents.append((pattern_list, tag['tag']))
        if tag['tag'] not in tags:
            tags.append(tag['tag'])

In [154]:
words = [Lmzr.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
tags = sorted(set(tags))

In [155]:
training = []

In [156]:
for document in documents:
  max_len = len(max(document, key=len))
  print(max_len)
  for pattern_list in document:
    bag = []
    pattern_list = [Lmzr.lemmatize(word.lower()) for word in pattern_list]
    for word in words:
        bag.append(1) if word in pattern_list else bag.append(0)

    output_row = [0] * len(tags)
    output_row[tags.index(document[1])] = 1


    binarydocument = [bag, output_row]
    training.append(binarydocument)

8
8
8
8
8
8
8
8
8
7
7
7
7
7
7
7
6
6
6
6
6
11
11
11
11
11
10
10
10
10
10
4
4
6
4
7
6
6
7
5
5
5
4
5
4
6
6
7
7
7
7
6
5
5
4
5
6
9
5
6
6
8
7
6
6
6
6
5
5
4
6
5
5
5
6
5
5
5
5
5
4
4
5
6
6
7
6
8
8
8
8
5


In [157]:
random.shuffle(training)
training = np.array(training, dtype=object)

In [158]:
train_patterns = np.array([np.array(x, dtype=np.float32) for x in training[:, 0]])
train_tags = np.array([np.array(x, dtype=np.float32) for x in training[:, 1]])

In [159]:
model = krs.Sequential()
model.add(krs.layers.Dense(48, input_shape=(np.shape(train_patterns[0])), activation='relu'))
model.add(krs.layers.Dense(28, activation='relu'))
model.add(krs.layers.Dense(28, activation='relu'))
model.add(krs.layers.Dense(24, activation='relu'))
model.add(krs.layers.Dense(24, activation='relu'))
model.add(krs.layers.Dense(len(train_tags[0]), activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [160]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_patterns, train_tags, epochs=80)

Epoch 1/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0885 - loss: 3.0445
Epoch 2/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1200 - loss: 3.0358
Epoch 3/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1386 - loss: 3.0290
Epoch 4/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1018 - loss: 3.0231
Epoch 5/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0924 - loss: 3.0141
Epoch 6/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1408 - loss: 3.0011
Epoch 7/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1145 - loss: 2.9913
Epoch 8/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1267 - loss: 2.9792 
Epoch 9/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1192 - loss: 2.9589 
Epoch 10/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1000 - loss: 2.9435
Epoch 11/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1200 - loss: 2.9071
Epoch 12/80
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1325 - loss: 2.8673
E

In [161]:
Acc, Loss = model.evaluate(train_patterns, train_tags)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7259 - loss: 0.6034  


In [162]:
print("Accuracy ---> ", train_patterns)
print("Loss ---> ", train_tags)

Accuracy --->  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
Loss --->  [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [163]:
def sentense_cleaner(sentense):
    sentense_words = nltk.word_tokenize(sentense)
    sentense_words = [Lmzr.lemmatize(word.lower()) for word in sentense_words]
    return sentense_words

In [164]:
def bag_words(sentense, words):
    sentense_words = sentense_cleaner(sentense)
    bag = [0] * len(words)

    for sentense_word in sentense_words:
       for i, word in enumerate(words):
         if word == sentense_word:
          bag[i] = 1
    return np.array(bag)

In [165]:
def predict_tag(sentense):
    bow = bag_words(sentense, words)
    res = np.array([np.array(x, dtype=np.float32) for x in bow])
    pred = np.argmax(model.predict(np.array([res])))
    tag = tags[pred]
    for intent in intents['intents']:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
    return response

In [166]:
!pip install gradio

In [167]:
import gradio as gr

In [168]:
iface = gr.Interface(
    inputs=gr.Textbox(lines=2, placeholder="Enter your message here..."),
    fn=predict_tag,
    outputs="text",
    title="FrostBite Chatbot",
    description="Interact with the chatbot and ask your questions.",
)

In [169]:
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64a4de8f3d8ec092ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
